In [11]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [2]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [12]:
#you can download the pretrained weights from the following link 
#https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
#or you can find the detailed documentation https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

from keras.models import model_from_json
model.load_weights('vgg_face_weights.h5')

In [4]:
def preprocess_image(image_path):
    #img = load_img(image_path, target_size=(224, 224))
    #img = img_to_array(img)
    #img = np.expand_dims(image_path, axis=0)
    img = preprocess_input(image_path)
    return img

In [5]:
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [6]:
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [11]:
import glob
epsilon = 0.40
images=glob.glob('dataset/*.jpg')
def verifyFace(img1):
    img1_representation = vgg_face_descriptor.predict(preprocess_image(img1))[0,:]
    cosine=[]
    faces=[]
    
    for i in range(len(images)):
        img2=cv2.imread(images[i])
        img2=cv2.resize(img2, (width, height))
        img2.shape
        img2=img2.reshape(1,224,224,3)
        img2_representation = vgg_face_descriptor.predict(preprocess_image(img2))[0,:]
    
        cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
        euclidean_distance = findEuclideanDistance(img1_representation, img2_representation)
    
        #print("Cosine similarity: ",cosine_similarity)
        #print("Euclidean distance: ",euclidean_distance)
        
        
        if(cosine_similarity < epsilon):
            cosine.append(cosine_similarity)
            faces.append(images[i])
            
            #return cosine_similarity
            #print("verified... they are same person")
        else:
            return cosine_similarity
            #print("unverified! they are not same person!")
        if len(cosine) !=0:
            pos = cosine.index(min(cosine))
            #print(pos)
            res = faces[pos]
        else:
            res =  0
    
        print("-----------------------------------------")
    return res






In [8]:
import cv2
import sys
import glob

faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
(width, height) = (224, 224)
img1=cv2.imread('dataset/321.jpeg')
img1=cv2.resize(img1, (width, height))
img1.shape
img1=img1.reshape(1,224,224,3)
video_capture = cv2.VideoCapture(1)

                 
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        frame,
        scaleFactor=1.1,
        minNeighbors=5
        #minSize=(160, 160),
        #flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        rrec = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        face = frame[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height))
        face_resize.shape
        face_resize = face_resize.reshape(1,224,224,3)
        text = verifyFace(face_resize, img1)
        text=str(text)
        cv2.putText(rrec,text,(100,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),1,cv2.LINE_AA)
        
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

TypeError: verifyFace() takes 1 positional argument but 2 were given

In [18]:
import cv2
import sys
import glob


faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')

(width, height) = (224, 224)

video_capture = cv2.VideoCapture(0)
idx=0
                 
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        frame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(224, 224),
        #flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        rrec = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        face = frame[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height))
        #face_resize.shape
        face_resize = face_resize.reshape(1,224,224,3)
        text = verifyFace(face_resize)
        text=str(text)
        if text=='donot':
            
            write_name = 'dataset/new face.'+str(idx)+'.jpg'
            cv2.imwrite(write_name, face)
            idx+=1
        cv2.putText(rrec,text,(100,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),1,cv2.LINE_AA)
        
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

[0.383620023727417, 0.2354246973991394, 0.3948202133178711, 0.5790610909461975, 0.38421642780303955, 0.3970634937286377, 0.2206088900566101, 0.45228129625320435, 0.38255053758621216, 0.1445363163948059, 0.39763033390045166, 0.07173305749893188, 0.3919040560722351, 0.4087962508201599, 0.3914046287536621]
['dataset/new face.4.jpg', 'dataset/gala1.jpg', 'dataset/new face.8.jpg', 'dataset/0004_01.jpg', 'dataset/Shamil.jpg', 'dataset/new face.7.jpg', 'dataset/User.2.2.jpg', 'dataset/new face.0.jpg', 'dataset/new face.3.jpg', 'dataset/321.jpg', 'dataset/new face.5.jpg', 'dataset/123.jpg', 'dataset/new face.6.jpg', 'dataset/new face.1.jpg', 'dataset/new face.2.jpg']
[0.3957972526550293, 0.2412150502204895, 0.40790945291519165, 0.570666640996933, 0.37744808197021484, 0.40562981367111206, 0.21573621034622192, 0.4730333089828491, 0.3970620632171631, 0.14061444997787476, 0.4067888855934143, 0.06627082824707031, 0.4022824764251709, 0.42716550827026367, 0.4068108797073364]
['dataset/new face.4.jpg'

In [17]:
import glob
import cv2
epsilon=0.2
images=glob.glob('dataset/*.jpg')
(width, height) = (224, 224)

def verifyFace(img1):
    img1_representation = vgg_face_descriptor.predict(preprocess_image(img1))[0,:]
    
    cosines=[]
    faces=[]
    for i in range(len(images)):
        img2=cv2.imread(images[i])
        img2=cv2.resize(img2, (width, height))
        img2.shape
        img2=img2.reshape(1,224,224,3)
        img2_representation = vgg_face_descriptor.predict(preprocess_image(img2))[0,:]

        cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
        cosines.append(cosine_similarity)
        
        #print(cosines)
        #print(images)
        
            
    #print(minimum)
    minimum = cosines.index(min(cosines))
    #print(cosines[minimum])
    print(cosines)
    print(images)
    #print('--------------------------------------------------------------------------')
    
    
    if float(cosines[minimum])<0.2:
        return images[minimum]
            
    if float(cosines[minimum])>0.2:
        return 'donot'
    
#return res
    
    

    
   
    #if(cosine_similarity < epsilon):
     #   print(cosine_similarity)
    
    

In [ ]:
import cv2
import sys
import glob


faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')

(width, height) = (224, 224)

video_capture = cv2.VideoCapture(0)
idx=0
                 
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        frame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(224, 224),
        #flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        rrec = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        face = frame[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height))
        #face_resize.shape
        face_resize = face_resize.reshape(1,224,224,3)
        text = verifyFace(face_resize)
        text=str(text)
        if text=='donot':
            
            write_name = 'dataset/new face.'+str(idx)+'.jpg'
            cv2.imwrite(write_name, face)
            idx+=1
        cv2.putText(rrec,text,(100,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),1,cv2.LINE_AA)
        
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

[0.4188094139099121, 0.2767372131347656, 0.4100835919380188, 0.5538826882839203, 0.38239938020706177, 0.39794373512268066, 0.18326318264007568, 0.5183283686637878, 0.4214453101158142, 0.12759196758270264, 0.40325498580932617, 0.08119088411331177, 0.40554356575012207, 0.4658434987068176, 0.43098169565200806]
['dataset/new face.4.jpg', 'dataset/gala1.jpg', 'dataset/new face.8.jpg', 'dataset/0004_01.jpg', 'dataset/Shamil.jpg', 'dataset/new face.7.jpg', 'dataset/User.2.2.jpg', 'dataset/new face.0.jpg', 'dataset/new face.3.jpg', 'dataset/321.jpg', 'dataset/new face.5.jpg', 'dataset/123.jpg', 'dataset/new face.6.jpg', 'dataset/new face.1.jpg', 'dataset/new face.2.jpg']


In [1]:
import glob
import cv2
epsilon=0.2
images=glob.glob('dataset/*.jpg')
(width, height) = (224, 224)

def verifyFace(img1):
    img1_representation = vgg_face_descriptor.predict(preprocess_image(img1))[0,:]
    
    cosines=[]
    faces=[]
    for i in range(len(images)):
        img2=cv2.imread(images[i])
        img2=cv2.resize(img2, (width, height))
        img2.shape
        img2=img2.reshape(1,224,224,3)
        img2_representation = vgg_face_descriptor.predict(preprocess_image(img2))[0,:]

        cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
        cosines.append(cosine_similarity)
        
        #print(cosines)
        #print(images)
        
            
    #print(minimum)
    minimum = cosines.index(min(cosines))
    #print(cosines[minimum])
    print(cosines)
    print(images)
    #print('--------------------------------------------------------------------------')
    
    
    if float(cosines[minimum])<0.2:
        return images[minimum]
            
    if float(cosines[minimum])>0.2:
        return 'donot'
    
#return res
    
    

    
   
    #if(cosine_similarity < epsilon):
     #   print(cosine_similarity)
    
    

In [10]:
import cv2
import sys
import glob

faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

(width, height) = (224, 224)
idx=0
                 
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    faces = faceCascade.detectMultiScale(
        frame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(224, 224)
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        rrec = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        face = frame[y:y + h, x:x + w] 
        
    face_resize = cv2.resize(face, (width, height))
    face_resize = face_resize.reshape(1,224,224,3)
    funct = func(face_resize)
    text='Kuanysh'
    cv2.putText(rrec,text,(x,y), cv2.FONT_HERSHEY_SIMPLEX, 1,(167,34,56),1,cv2.LINE_AA)
        
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

[0.2347462773323059, 0.5857961177825928, 0.41034621000289917, 0.22024130821228027, 0.1422436237335205, 0.0862511396408081]
['dataset/gala1.jpg', 'dataset/0004_01.jpg', 'dataset/Shamil.jpg', 'dataset/User.2.2.jpg', 'dataset/321.jpg', 'dataset/123.jpg']
[0.23466253280639648, 0.5858205556869507, 0.4099884629249573, 0.22009211778640747, 0.14229649305343628, 0.08657091856002808]
['dataset/gala1.jpg', 'dataset/0004_01.jpg', 'dataset/Shamil.jpg', 'dataset/User.2.2.jpg', 'dataset/321.jpg', 'dataset/123.jpg']
[0.23458731174468994, 0.5858084559440613, 0.409792959690094, 0.22003644704818726, 0.1423349380493164, 0.08675980567932129]
['dataset/gala1.jpg', 'dataset/0004_01.jpg', 'dataset/Shamil.jpg', 'dataset/User.2.2.jpg', 'dataset/321.jpg', 'dataset/123.jpg']
[0.2345280647277832, 0.5858409702777863, 0.4096517562866211, 0.2199879288673401, 0.14235544204711914, 0.08688229322433472]
['dataset/gala1.jpg', 'dataset/0004_01.jpg', 'dataset/Shamil.jpg', 'dataset/User.2.2.jpg', 'dataset/321.jpg', 'dataset/

In [9]:
import glob
import cv2
epsilon=0.2
images=glob.glob('dataset/*.jpg')
(width, height) = (224, 224)

def func(img1):
    img1_representation = vgg_face_descriptor.predict(preprocess_image(img1))[0,:]
    
    cosines=[]
    faces=[]
    for i in range(len(images)):
        img2=cv2.imread(images[i])
        img2=cv2.resize(img2, (width, height))
        img2.shape
        img2=img2.reshape(1,224,224,3)
        img2_representation = vgg_face_descriptor.predict(preprocess_image(img2))[0,:]

        cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
        cosines.append(cosine_similarity)
    print(cosines)
    print(images)